In [3]:
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
import keras
from tensorflow.keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator

In [4]:
training_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.12,
    height_shift_range = 0.12,
    width_shift_range = 0.12,
    horizontal_flip = True
)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = training_data_gen.flow_from_directory('./data/images/Images',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')
test_generator = test_data_gen.flow_from_directory('./data/test',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 16666 images belonging to 120 classes.
Found 3911 images belonging to 120 classes.


In [6]:
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False

In [7]:
inputs = keras.Input(shape=(300, 300, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

model = Model(base_model.input, predictions)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1/conv[0][0]']         

In [8]:
model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [13]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='accuracy',
    patience = 1,
    restore_best_weights = True
)
model.fit(train_generator,
                        epochs=10,
                        validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/10
521/521 [==============================] - 3207s 6s/step - loss: 3.1478 - accuracy: 0.3470 - val_loss: 1.5510 - val_accuracy: 0.6505
Epoch 2/10
521/521 [==============================] - 1799s 3s/step - loss: 1.2511 - accuracy: 0.6728 - val_loss: 0.9320 - val_accuracy: 0.7405
Epoch 3/10
521/521 [==============================] - 1780s 3s/step - loss: 0.9151 - accuracy: 0.7361 - val_loss: 0.8086 - val_accuracy: 0.7579
Epoch 4/10
521/521 [==============================] - 1793s 3s/step - loss: 0.7871 - accuracy: 0.7706 - val_loss: 0.7431 - val_accuracy: 0.7727
Epoch 5/10
521/521 [==============================] - 1815s 3s/step - loss: 0.7134 - accuracy: 0.7823 - val_loss: 0.6878 - val_accuracy: 0.7847
Epoch 6/10
521/521 [==============================] - 1798s 3s/step - loss: 0.6596 - accuracy: 0.7967 - val_loss: 0.6690 - val_accuracy: 0.7937
Epoch 7/10
521/521 [==============================] - 1801s 3s/step - loss: 0.6233 - accuracy: 0.8094 - val_loss: 0.6601 - val_accuracy:

In [14]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'\nTest accuracy: {test_acc}')

123/123 - 332s - loss: 0.6312 - accuracy: 0.8008 - 332s/epoch - 3s/step

Test accuracy: 0.8008182048797607


In [16]:
model.save('NN/stanford_dog_densenet.h5')